<a href="https://colab.research.google.com/github/Jana-Alrzoog/2025_GP_28/blob/main/masar-sim/notebooks/masar_modifier_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The primary objective of this notebook is to formulate and validate the logic for computing demand modifiers. These are dimensionless factors that quantify the expected deviation from a baseline demand level due to a set of external variables.

The function developed here does not generate the demand itself, but rather calculates a multiplier to adjust a pre-existing baseline. The variables considered include:

Inherent attributes: Station size and passenger capacity.

*   Inherent attributes: Station info and passenger capacity.
*   Temporal factors: Day of the week (weekday or weekend).
*   Environmental conditions: Pre-defined weather patterns.
*   Scheduled occurrences: Public holidays and special events.





Temporal factors: Day of the week (weekday vs. weekend).

Environmental conditions: Pre-defined weather patterns.

Scheduled occurrences: Public holidays and special events.

The output is a single numerical multiplier. A value of 1.0 signifies normal conditions, while values above or below 1.0 indicate an expected increase or decrease in passenger demand, respectively.

In [ ]:
!git clone https://github.com/Jana-Alrzoog/2025_GP_28.git
%cd 2025_GP_28


Cloning into '2025_GP_28'...
remote: Enumerating objects: 431, done.
remote: Counting objects: 100% (430/430), done.
remote: Compressing objects: 100% (316/316), done.
remote: Total 431 (delta 126), reused 303 (delta 67), pack-reused 1 (from 1)
Receiving objects: 100% (431/431), 1.67 MiB | 6.49 MiB/s, done.
Resolving deltas: 100% (126/126), done.
/content/2025_GP_28/2025_GP_28/2025_GP_28


In [ ]:
!ls


analysis_options.yaml  ios    masar-sim     replacements.txt
android		       lib    pubspec.lock  test
assets		       linux  pubspec.yaml  web
AUTHORS.txt	       macos  README.md     windows


In [ ]:
!pip install pyyaml pandas

In [ ]:
import yaml, json, csv
from datetime import datetime

# تحديد المسارات
seed_path = "/content/2025_GP_28/masar-sim/data/seeds"
config_path = "/content/2025_GP_28/masar-sim/sims/00_config.yaml"

# تحميل الإعدادات
with open(config_path) as f:
    config = yaml.safe_load(f)

# تحميل ملفات seeds
seeds = {}
with open(f"{seed_path}/stations.json") as f:
    seeds["stations"] = json.load(f)
with open(f"{seed_path}/weather_patterns.json") as f:
    seeds["weather"] = json.load(f)
with open(f"{seed_path}/calendar_events.csv") as f:
    seeds["events"] = list(csv.DictReader(f))
with open(f"{seed_path}/holidays.csv") as f:
    seeds["holidays"] = list(csv.DictReader(f))

print(" Loaded successfully")
print("Stations:", len(seeds["stations"]))
print("Events:", len(seeds["events"]))
print("Holidays:", len(seeds["holidays"]))


 Loaded successfully
Stations: 6
Events: 10
Holidays: 5


Here is the main function that integrates all variables to produce the final modifier. It accepts a timestamp and station identifier as input, calculates the individual multipliers for each factor (weekend, weather, event, holiday), and then computes their product to capture the cumulative effect. The function returns a dictionary containing the final computed modifie

In [ ]:
def _match_station(sta, key):
    k = str(key).upper()
    return str(sta.get("station_id","")).upper() == k or str(sta.get("code","")).upper() == k

def _get_station(seeds, key):
    for s in seeds["stations"]:
        if _match_station(s, key):
            return s
    raise ValueError(f"Station not found: {key}")

def _station_scale_from_capacity(stations, rec):
    caps = [r.get("capacity_platform") for r in stations if isinstance(r.get("capacity_platform"), (int,float))]
    mean_cap = (sum(caps)/len(caps)) if caps else 1500.0
    return float(rec.get("capacity_platform", mean_cap))/float(mean_cap) if mean_cap>0 else 1.0

def compute_demand_modifier(ts, station_key, seeds, config):
    date_str = ts.date().isoformat()
    weekday  = ts.weekday()
    st = _get_station(seeds, station_key)
    station_scale = _station_scale_from_capacity(seeds["stations"], st)

    weekend_mult = float(config["multipliers"]["weekend"]) if weekday in [4,5] else 1.0
    w = seeds["weather"].get(date_str, {"condition":"Sunny"})
    weather_cond = w.get("condition","Sunny")
    weather_mult = float(config["multipliers"]["weather"].get(weather_cond, 1.0))

    # match events by either station_id or code
    sid = str(st.get("station_id","")).upper()
    scode = str(st.get("code","")).upper()
    event_mult = 1.0
    for ev in seeds["events"]:
        ev_station = str(ev.get("station_id") or ev.get("station") or ev.get("station_code") or "").upper()
        if ev.get("date")==date_str and ev_station in {sid, scode}:
            et = ev.get("event_type","Other")
            event_mult = max(event_mult, float(config["events"].get(et, 1.0)))

    holiday_mult = float(config["multipliers"]["holiday"])
    for hol in seeds["holidays"]:
        if hol.get("date")==date_str:
            try: holiday_mult = float(hol.get("demand_modifier", holiday_mult))
            except: pass

    final = station_scale * weekend_mult * weather_mult * event_mult * holiday_mult
    return {"station": scode or sid, "date": date_str, "weather": weather_cond,
            "station_scale": station_scale, "weekend_mult": weekend_mult,
            "weather_mult": weather_mult, "event_mult": event_mult,
            "holiday_mult": holiday_mult, "final_demand_modifier": final}


 This section serves to test and validate the core logic. A specific scenario(  KAFD station on a public holiday ) is used to call the compute_demand_modifier function. The output is then inspected to confirm that the logic correctly identifies the relevant conditions and calculates a factually consistent modifier.

 and the result we had is a detailed forecast predicting that passenger demand at the KAFD station on Saudi National Day (September 23, 2025) will be more than double the normal amount.

The final prediction was 2.16, meaning the demand is expected to be about 217% of a standard day. This number was calculated by multiplying several factors together.



In [ ]:
from datetime import datetime

# Example: compute modifier for KAFD station on National Day
ts = datetime(2025, 9, 23, 9, 0)  # time and date
station_key = "KAFD"               # works with either "KAFD" or "S1"

result = compute_demand_modifier(ts, station_key, seeds, config)
result


{'station': 'KAFD',
 'date': '2025-09-23',
 'weather': 'Sunny',
 'station_scale': 1.0843373493975905,
 'weekend_mult': 1.0,
 'weather_mult': 1.0,
 'event_mult': 1.0,
 'holiday_mult': 2.0,
 'final_demand_modifier': 2.168674698795181}